### Practice 0808

In [20]:
# using resample to group rows by chunks of time
import pandas as pd
import numpy as np

# create date range
time_index = pd.date_range('06/06/2017', periods=100000, freq='30S')

# create dataframe
dataframe = pd.DataFrame(index=time_index)

In [21]:
# create column of random values
dataframe['sale_amount'] = np.random.randint(1, 10, 100000)

# group rows by week, calculate sum per week
dataframe.resample('W').sum()

,sale_amount
2017-06-11,86288
2017-06-18,101052
2017-06-25,100904
2017-07-02,100825
2017-07-09,100736
2017-07-16,10463
